# ECO6128 Tutorial - Text Similarity

Introducing cosine and Jaccard similarity.

Created by *Xinghao YU*, March 18th, 2023

*Copyright@Chinese University of Hong Kong, Shenzhen*

In [ ]:
import jieba
from sklearn.metrics.pairwise import cosine_similarity
corpus = []
with open('./hotel.txt') as f:
    for line in f:
        corpus.append(''.join([word for word in jieba.lcut(line.strip())]))

## Cosine similarity

In [ ]:
class CosineSimilarity(object):
    def __init__(self, content_x1, content_y2):
        self.s1 = content_x1
        self.s2 = content_y2

    @staticmethod
    def extract_keyword(content):  # 提取关键词
        # 切割
        seg = [i for i in jieba.cut(content, cut_all=True) if i != '']
        # 提取关键词
        keywords = jieba.analyse.extract_tags("|".join(seg), topK=200, withWeight=False)
        return keywords

    @staticmethod
    def one_hot(word_dict, keywords):  # oneHot编码
        # cut_code = [word_dict[word] for word in keywords]
        cut_code = [0]*len(word_dict)
        for word in keywords:
            cut_code[word_dict[word]] += 1
        return cut_code

    def main(self):
        # 去除停用词
        jieba.analyse.set_stop_words('./stop_word.txt')

        # 提取关键词
        keywords1 = self.extract_keyword(self.s1)
        keywords2 = self.extract_keyword(self.s2)
        
        # 词的并集
        union = set(keywords1).union(set(keywords2))
        
        # 编码
        word_dict = {}
        i = 0
        for word in union:
            word_dict[word] = i
            i += 1
        #print(word_dict)
        
        # oneHot编码
        s1_cut_code = self.one_hot(word_dict, keywords1)
        s2_cut_code = self.one_hot(word_dict, keywords2)
        
        # 余弦相似度计算
        sample = [s1_cut_code, s2_cut_code]
        # print(sample)
        
        # 除零处理
        try:
            sim = cosine_similarity(sample)
            return sim[1][0]
        except Exception as e:
            print(e)
            return 0.0


# 测试
if __name__ == '__main__':
    for i in range(0, len(corpus)-1):
        content_x, content_y = corpus[i], corpus[i+1]
        similarity = CosineSimilarity(content_x, content_y)
        similarity_result = similarity.main()
        print('%d, %d, Similarity: %.2f%%' % (i, i+1, similarity_result * 100))

## Jaccard similarity

In [ ]:
class JaccardSimilarity(object):
    def __init__(self, content_x1, content_y2):
        self.s1 = content_x1
        self.s2 = content_y2

    @staticmethod
    def extract_keyword(content):  # 提取关键词
        # 切割
        seg = [i for i in jieba.cut(content, cut_all=True) if i != '']
        # 提取关键词
        keywords = jieba.analyse.extract_tags("|".join(seg), topK=200, withWeight=False)
        return keywords

    def main(self):
        # 去除停用词
        jieba.analyse.set_stop_words('./stop_word.txt')

        # 分词与关键词提取
        keywords_x = self.extract_keyword(self.s1)
        keywords_y = self.extract_keyword(self.s2)

        # jaccard相似度计算
        intersection = len(list(set(keywords_x).intersection(set(keywords_y))))
        union = len(list(set(keywords_x).union(set(keywords_y))))
        # 除零处理
        sim = float(intersection)/union if union != 0 else 0
        return sim


# 测试
if __name__ == '__main__':
    for i in range(0, len(corpus)-1):
        content_x, content_y = corpus[i], corpus[i+1]
        similarity = JaccardSimilarity(content_x, content_y)
        similarity_result = similarity.main()
        print('%d, %d, Similarity: %.2f%%' % (i, i+1, similarity_result * 100))